In [58]:
import json
import pickle
import time
from tqdm import tqdm

valfile = "/home/cse/btech/cs1200389/MatLlama/MatLLaMA/datasets/downstream/val/val.jsonl"
def load_jsonl(path):
    with open(path, 'r') as f:
        a = f.readlines()
        g = [json.loads(i) for i in a]
    return g
val_ds = load_jsonl(valfile)
doping_test = load_jsonl('/home/cse/btech/cs1200389/MatLlama/MatLLaMA/datasets/ift/final_ift/doping_test.jsonl')
mof1_test = load_jsonl('/home/cse/btech/cs1200389/MatLlama/MatLLaMA/datasets/ift/final_ift/mof1_test.jsonl')
mof2_test = load_jsonl('/home/cse/btech/cs1200389/MatLlama/MatLLaMA/datasets/ift/final_ift/mof2_test.jsonl')
with open("/home/cse/btech/cs1200389/MatLlama/MatLLaMA/datasets/ift/final_ift/discomat_dense_test.jsonl", 'r') as f:
    discomat_test = [json.loads(line) for line in f.readlines()]


In [59]:
import os
# os.environ['http_proxy'] ="10.10.78.22:3128"
# os.environ['https_proxy'] ="10.10.78.22:3128"


In [ ]:
import anthropic
apii ="your api key"
client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=api,
)

def get_prompt(input_dict):
    system_prompt = input_dict['system']
    prompt = input_dict['question']
    return system_prompt, prompt

def get_output(input_dict, additional = ""):
    sys, prompt = get_prompt(input_dict)
    message = client.messages.create(
    model="claude-3-opus-20240229", 
    # model = "claude-3-haiku-20240307",
    max_tokens=2000,
    temperature=0,
    top_p=0.9,
    system = sys + additional,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "question\n" + prompt
                }
            ]
        }
    ]
)
    return message.content[0].text #return the text content of the output response

outname = "claude_3_opus_val"

In [47]:
# with open(outname + "_downstream.pkl", "rb") as f:
#     downstream_outputs = pickle.load(f)
# print(len(check))

12336


## Downstream run

In [54]:
# downstream_outs = sorted(downstream_outputs, key = lambda x : x[0])

In [55]:
# actual = [downstream_outs[0]]
# for i in range(1,len(downstream_outs)):
#     if(downstream_outs[i-1][0] == downstream_outs[i][0]):
#         actual[-1] = downstream_outs[i]
#     else:
#         actual.append(downstream_outs[i])

In [ ]:
import pickle
import time
from tqdm import tqdm

downstream_outputs = []
exceptions = []
start_time = time.time()

# for idx in tqdm(range(5001,len(val_ds))):
idx = 0
failures = 0
start = time.time()
while idx < len(val_ds):
    try:
        out = get_output(val_ds[idx], additional = ". if JSON format is asked then do not output anything aside from the json string")
        # downstream_outputs.append((idx, out))
        downstream_outputs.append((idx, out))
        idx += 1
        failures = 0 #resetting failures.
        print("success for ", idx, " rate = ", (time.time() - start)/(idx), end = "          \r")
    except Exception as e:
        print("failed at ", idx, " because\n", e)
        failures += 1
        if(failures >= 6):
            downstream_outputs.append((idx, "")) 
            print(f"IDX {idx} failed to calculate, setting to ''")
            failures = 0
            idx += 1
        else:
            time.sleep(5)
            continue    #sleep and then redo the same index.
    if(idx % 1000 == 0):
        with open(outname + "_downstream.pkl", "wb") as f:
            pickle.dump(downstream_outputs, f) #saving every 1000 iterations incase something goes wrong and generation has to stop in the middle.
with open(outname + "_downstream.pkl", "wb") as f:
    pickle.dump(downstream_outputs, f)

failed at  20  because=  2.7508530735969545          
 Error code: 500 - {'type': 'error', 'error': {'type': 'api_error', 'message': 'Internal server error'}}


## SIE Doping mof1 mof2 run

In [ ]:
doping_outputs = []
exceptions = []
import time
# for idx in tqdm(range(len(doping_test))):
idx = 0
start = time.time()
failures = 0
while(idx < len(doping_test)):
    try:
        out = get_output(doping_test[idx], additional = "do not output any additional text")
        doping_outputs.append((idx, out))
        idx += 1
        print("at index ", idx, " at time: ", time.time() - start, "rate = ", (time.time() -start)/idx, "s/out", end = "   \r")
        failures = 0
    except Exception as e:
        print("failed at ", idx, " because\n", e)
        exceptions.append((idx, e))
        failures += 1
        if(failures >= 6):
            failures = 0
            doping_outputs.append((idx,""))
            idx += 1
            print(f"setting {idx} to ''\n")
        time.sleep(2);
        # No updating the idx incase we encounter an error because it is likely a rate limit error.  
with open(outname + "_doping.pkl", "wb") as f:
    pickle.dump(doping_outputs, f)

In [26]:
mof1_outputs = []
exceptions = []
idx = 0;
start = time.time()
while(idx < len(mof1_test)):
    print("at index ", idx, " at time: ", time.time() - start, "rate = ", (idx/(time.time() -start)), end = "   \r")
    try:
        out = get_output(mof1_test[idx],additional = "do not output any additional text")
        mof1_outputs.append((idx, out))
        idx += 1
    except Exception as e:
        print("failed at ", idx, " because\n", e)
        exceptions.append((idx, e))
        time.sleep(5)
with open(outname + "_mof1.pkl", "wb") as f:
    pickle.dump(mof1_outputs, f)

failed at  7  because  17.69264578819275 rate =  0.39564458124727514    
 Error code: 500 - {'type': 'error', 'error': {'type': 'api_error', 'message': 'Internal server error'}}
failed at  56  because  782.1214780807495 rate =  0.07160013050017981    
 Error code: 500 - {'type': 'error', 'error': {'type': 'api_error', 'message': 'Internal server error'}}


In [ ]:
# with open("../llamat2_chat_train_2epochs_full_ex_mof1_test.pkl", "rb") as f:
#     check= pickle.load(f)

In [14]:
mof2_outputs = []
exceptions = []
idx = 0;
start = time.time()
while(idx < len(mof2_test)):
    print("at index ", idx, " at time: ", time.time() - start, "rate = ", (idx/(time.time() -start)), end = "   \r")
    try:
        out = get_output(mof2_test[idx],additional = "do not output any additional text")
        mof2_outputs.append((idx, out))
        idx += 1
    except Exception as e:
        print("failed at ", idx, " because\n", e)
        exceptions.append((idx, e))
with open(outname + "_mof2.pkl", "wb") as f:
    pickle.dump(mof2_outputs, f)

## Discomat run

In [15]:
discomat_outputs = []
idx = 0;
start = time.time()
while(idx < len(discomat_test)):
    print("at index ", idx, " at time: ", time.time() - start, "rate = ", (idx/(time.time() -start)), end = "   \r")
    try:
        out = get_output(discomat_test[idx],additional = "do not output any additional text")
        discomat_outputs.append((idx, out))
        idx += 1
    except Exception as e:
        print("failed at ", idx, " because\n", e)
        exceptions.append((idx, e))
with open(outname + "_discomat.pkl", "wb") as f:
    pickle.dump(discomat_outputs, f)

failed at  697  because  7371.856674671173 rate =  0.09454877253713312     
 Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}


In [17]:
print("done")

done
